In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


df = pd.read_csv(r"C:\Users\matti\OneDrive\Thesis\stats\aggregate_metrics_25runs.csv", sep=",")  

df[["Mean", "Std"]] = df[df.columns[1]].str.extract(r"([\d.]+)\s*±\s*([\d.]+)").astype(float)

n = 25  #number of runs
z = 1.96  #95% confidence interval

#Calculate
df["Margin of Error"] = z * df["Std"] / np.sqrt(n)
df["CI Lower"] = df["Mean"] - df["Margin of Error"]
df["CI Upper"] = df["Mean"] + df["Margin of Error"]
df["95% CI"] = df.apply(lambda row: f"[{row["CI Lower"]:.4f}, {row["CI Upper"]:.4f}]", axis=1)

df.to_csv("aggregate_metrics_with_ci.csv", index=False)



In [ ]:
import pandas as pd
from scipy.stats import shapiro, ttest_rel, wilcoxon
import itertools

df = pd.read_csv(r"C:\Users\matti\OneDrive\Thesis\stats\all_metrics_25runs (1).csv", sep=",")  

models = ["lstm", "gru", "xgb", "stack_lr", "stack_nn"]
metrics = ["rmse", "mae", "mape", "mda"]

results = []

for metric in metrics:
    for model1, model2 in itertools.combinations(models, 2):
        col1 = f"{model1}_{metric}"
        col2 = f"{model2}_{metric}"
        diffs = df[col1] - df[col2]
        _, p_norm = shapiro(diffs)
        is_normal = p_norm > 0.05

        if is_normal:
            test_used = "Paired t-test"
            stat, p_val = ttest_rel(df[col1], df[col2])
        else:
            test_used = "Wilcoxon"
            try:
                stat, p_val = wilcoxon(df[col1], df[col2])
            except ValueError:
                stat, p_val = None, None #Only happens if all are 0

        results.append({
            "Comparison": f"{model1} vs {model2}",
            "Metric": metric,
            "Normality p": round(p_norm, 4),
            "Normal Dist?": "Yes" if is_normal else "No",
            "Test Used": test_used,
            "p-value": round(p_val, 4) if p_val is not None else "NA",
            "Statistically Significant?": "Yes" if p_val is not None and p_val < 0.05 else "No"
        })

result_df = pd.DataFrame(results)

result_df.to_csv("correlations.csv", index=False)


